In [35]:
import pandas as pd
import seaborn as sns
import numpy as np
import matplotlib.pyplot as plt
import matplotlib
import datetime
from scipy import stats
from statsmodels.stats.anova import anova_lm
from statsmodels.formula.api import ols
import statsmodels.api as sm
import statsmodels.formula.api as smf
from statsmodels.nonparametric.smoothers_lowess import lowess
from Bio.SeqIO.FastaIO import SimpleFastaParser
import os
from upsetplot import UpSet

custom_params = {"axes.spines.right": False, "axes.spines.top": False}
sns.set_theme(style="ticks", rc=custom_params)

#run_type = "dry"
run_type = "wet"


new_day = datetime.datetime.now().strftime("%Y%m%d")

palette = {"Control":"#000000", "Rapa Adult":"#5a9f68", 
           "Rapa Early":"#4170b0", "Rapa Late":"#c34e0d"}

path = "../analysis/plots/expression/"

In [36]:
df = pd.read_csv("../analysis/CLEAN_ASVs_counts_merged.tsv", sep = "\t")
df.columns = ["Sample_ID"] + list(df.columns[1:])
metadata = pd.read_csv("../metadata_rapaInter.csv")

In [37]:
# Sort metadata according to columnames to help understand groups
sorter = list(df.columns[1:])
metadata.Sample_ID = metadata.Sample_ID.astype("category")
metadata.Sample_ID = metadata.Sample_ID.cat.set_categories(sorter)
metadata = metadata.sort_values(["Sample_ID"])
metadata.set_index("Sample_ID", inplace = True)

df.set_index("Sample_ID", inplace = True)

# Transpose df and merge with metadata
mer = pd.merge(df.T, metadata, left_index = True, right_index = True)

,ASV_1,ASV_2,ASV_3,ASV_4,ASV_5,ASV_6,ASV_7,ASV_8,ASV_9,ASV_10,...,ASV_4092,ASV_4093,ASV_4094,ASV_4095,ASV_4096,Mouse_ID,Sex,Treatment_SG,Treatment,Age
Plate1_A1,16520,9898,474,3754,3977,1587,1017,1484,2009,0,...,0,0,0,0,0,TNES-BK48,f,Rapamycin conti (RC),Rapa Adult,Old
Plate1_A10,14438,8690,2557,4349,4937,2110,3365,4770,1802,1111,...,0,0,0,0,0,TNES-BH32,f,Rapamycin conti (RC),Rapa Adult,Young
Plate1_A11,55,26,45,63,76,378,4,177,2035,19529,...,0,0,0,0,0,TNES-BM39,f,Control,Control,Old
Plate1_A12,16,7,1932,334,309,682,417,930,100,12127,...,0,0,0,0,0,TNES-BS28,m,Rapa memory Early (RM E),Rapa Early,Mid
Plate1_A2,7,0,989,1950,2075,1242,1709,98,78,14536,...,0,0,0,0,0,TNES-BJ14,f,Rapamycin conti (RC),Rapa Adult,Mid
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Plate4_F1,8150,4718,191,5,5,8,1339,2234,4270,26,...,0,0,0,0,0,TNES-BH57,f,Rapa memory Late (RM L),Rapa Late,Old
Plate4_F2,42340,26130,15,0,8,518,1107,3366,2453,0,...,0,0,0,0,0,TNES-BI53,f,Rapa memory Late (RM L),Rapa Late,Old
Plate4_F3,9001,5138,1635,4714,4775,2148,3234,244,517,2,...,5,5,5,5,5,TNES-BR49,m,Rapamycin conti (RC),Rapa Adult,Mid
Plate4_G1,14284,8906,3890,54,81,2534,1963,2245,5377,4,...,0,0,0,0,0,TNES-BS14,m,Control,Control,Old
